En este notebook se realiza una segunda aproximación utilizando imputación simple de valores faltantes y un modelo de árbol de decisión sin codificación one-hot, para las variables categóricas se utiliza codificación ordinal directa (Label Encoding). Esta aproximación permite ejecutar el modelo rápidamente con resultados interpretables



In [ ]:
#Carga datos desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


train_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/train.csv'
test_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/test.csv'


df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Elimina filas sin etiqueta
df_train = df_train.dropna(subset=['RENDIMIENTO_GLOBAL'])

# Imputa valores nulos
for df in [df_train, df_test]:
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('Desconocido')
        else:
            df[col] = df[col].fillna(df[col].median())

# Guarda ID del test para el archivo final
test_ids = df_test['ID']

# Codifica variables categóricas con control de valores nuevos
cat_cols = df_train.select_dtypes(include='object').columns.tolist()
cat_cols.remove('RENDIMIENTO_GLOBAL')

le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])


    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    df_test[col] = df_test[col].map(mapping).fillna(-1).astype(int)

    le_dict[col] = le

# Codifica la variable objetivo
target_le = LabelEncoder()
y = target_le.fit_transform(df_train['RENDIMIENTO_GLOBAL'])

X = df_train.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
X_test = df_test.drop(columns=['ID'])

# Entrena modelo de árbol de decisión
model = DecisionTreeClassifier(max_depth=6, random_state=42)
model.fit(X, y)

# Predice y decodifica etiquetas
preds_encoded = model.predict(X_test)
preds = target_le.inverse_transform(preds_encoded)

# Crea archivo csv para conocer el resultado del modelo
submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': preds})
submission.to_csv('submission.csv', index=False)







Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
